In [193]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys
from cStringIO import StringIO

import numpy as np
import requests

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')
if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [2]:
for module in ['hosts', 'targeting', 'mmthecto']:
    if module in globals():
        reload(globals()[module])
    else:
        globals()[module] = __import__(module)
g = targeting.get_gama() #re-caches the gama catalo

In [186]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [45]:
from astropy import units as u

from astropy.table import Table
from astropy import table
from astropy.coordinates import SkyCoord

In [5]:
# this version includes those both with *and* without incomplete data
mltab = Table.read('catalogs/SAGA.objid.noclean.PROBS_WISE_PROBS.sept28.fits.gz')
mltab

<Table masked=True length=1689331>
      OBJID_1             RA      ... BEST_GUESS_CLASS_WISE
       int64           float64    ...         int64        
------------------- ------------- ... ---------------------
1237645941824356440 40.2877964113 ...                     0
1237645941824356442 40.2745834514 ...                     0
1237645941824356481 40.2721052363 ...                     0
1237645941824356483 40.2712777717 ...                    --
1237645941824356497 40.2846730128 ...                     0
1237645941824356500 40.2881452154 ...                     0
1237645941824356501 40.2854134086 ...                    --
1237645941824356546 40.2730041845 ...                     0
1237645941824356597  40.287315948 ...                     0
1237645941824356668 40.2778355788 ...                     0
                ...           ... ...                   ...
1237680297281716962 1.44656721936 ...                    --
1237680297281717028 1.47648597721 ...                     0
1237680297281717037 1.47825579475 ...                     0
1237680297281717047 1.48462184179 ...                     0
1237680297281717051 1.48591854703 ...                     0
1237680297281717063 1.49105042578 ...                     0
1237680297281717067 1.49284998125 ...                     0
1237680297281717150 1.38641505958 ...                    --
1237680297281781900 1.49852208005 ...                     0
1237680297281782227 1.50438437016 ...                    --

In [6]:
allspec = Table.read(os.environ['HOME']+'/Dropbox/SAGA/data/allspectaken_v4.fits')
allsats = allspec[(allspec['SATS']==1)&(allspec['REMOVE']==-1)]
allsats

<Table masked=True length=121>
       objID              ra           dec       ... ZQUALITY SPEC_REPEAT
       int64           float64       float64     ... float64   string384 
------------------- ------------- -------------- ... -------- -----------
1237648702984683605 225.005401226 -1.09130226734 ...      4.0        SDSS
1237650372092690464  180.29544216  -1.2976838319 ...      4.0            
1237662524159557759 195.103918476  13.6702528117 ...      4.0        SDSS
1237662525233037420 194.572104136  14.5571518823 ...      4.0        SDSS
1237662196143947955 206.062030556  41.6773969113 ...      4.0        SDSS
1237662307796910232  206.19066673     41.5567177 ...      4.0        SDSS
1237662300829384713 228.771613868  42.2094568926 ...      4.0        SDSS
1237655369832267945  217.94533575  58.0087096127 ...      4.0            
1237664871897039021 141.022041382  34.6465641753 ...      4.0        SDSS
1237664871897170083 141.506228227  34.6534021854 ...      4.0        SDSS
                ...           ...            ... ...      ...         ...
1237664669514662021  171.11620079   39.814084535 ...      4.0        SDSS
1237662226210160806 170.457771868   40.001812079 ...      4.0        SDSS
1237664670051336378 170.402482545  40.1559946711 ...      4.0        SDSS
1237664669514596560 170.778945018  39.9567281033 ...      4.0        SDSS
1237660765378248765 137.962155506  34.9637406847 ...      4.0        SDSS
1237660765378314404 138.184974166  34.9255496114 ...      4.0        SDSS
1237660765915316352 138.040930283  35.5376147162 ...      4.0        SDSS
1237667255616143515  158.04324634  28.4830569538 ...      4.0        SDSS
1237667287812735027 157.778274298  28.7966447953 ...      4.0        SDSS
1237665367429677221 158.712397798  28.6638533288 ...      4.0        SDSS

In [7]:
# info on all the hosts to decide which to investigate further
nsaid2name = {}
for row in allsats:
    nsaid2name[row['HOST_NSAID']] = row['HOST_SAGA_NAME']
    
Table(rows=[(i, 
         nsaid2name[i], 
         np.sum(allsats['HOST_NSAID']==i), 
         np.sum((allsats['HOST_NSAID']==i)&(allsats['MASKNAME']!='sdss')))
        for i in np.unique(allsats['HOST_NSAID'])],
      names=['NSAID', 'host name', 'n_sats', 'n_sats_not_sdss']).pprint(max_lines=1000)

NSAID  host name n_sats n_sats_not_sdss
------ --------- ------ ---------------
    32                1               0
 16235                3               0
 16559                2               0
 33446  StarTrek      2               0
 61945      AnaK      2               0
 69028                3               1
 85746  MobyDick      1               1
 94217                1               0
135129                5               0
135440                1               0
135667   Beowulf      8               4
135739                3               1
135879                5               2
137625                3               0
138201                5               0
138375                3               1
138642                2               0
139281                4               0
140065                3               1
140301                2               1
140458                1               0
140594     Alice      1               1
141465                1               0


# Odyssey it is, then...

In [26]:
ody = hosts.odyssey

In [12]:
odycat = Table.read('catalogs/base_sql_nsa147100.fits.gz')
odycat

<Table masked=False length=116533>
       objID              ra           dec      ... ZQUALITY SPEC_REPEAT
       int64           float64       float64    ... float64   string376 
------------------- ------------- ------------- ... -------- -----------
1237662698115433791 248.118777512 19.8004829221 ...     -1.0            
1237662698115433225 248.122956558   19.84698244 ...     -1.0            
1237662698115433190 248.069496538 19.7887422658 ...     -1.0            
1237662698115433217 248.076445205 19.7864371366 ...     -1.0            
1237662698115433272 248.130562637 19.8317867144 ...     -1.0            
1237662698115433729 248.065855461 19.7910845264 ...     -1.0            
1237662698115433063 248.111859856 19.7906770758 ...     -1.0            
1237662698115433191 248.068663185 19.7882256128 ...     -1.0            
1237662698115433215 248.121799658  19.852698119 ...     -1.0            
1237662698115433282 248.100575783 19.7873935706 ...     -1.0            
                ...           ...           ... ...      ...         ...
1237662225166370117 248.990193165 20.3182059911 ...     -1.0            
1237662225166369828 248.959592913  20.393809806 ...     -1.0            
1237662225166435969 249.063393776 20.2152671969 ...     -1.0            
1237662225166435433 249.047164519 20.2100642873 ...     -1.0            
1237662225166369177 248.950740426 20.3764197577 ...     -1.0            
1237662225166369834  248.96111865 20.3912783233 ...     -1.0            
1237662225166436137 249.041582426 20.2671017685 ...     -1.0            
1237662225166369739 248.947400955 20.4144193816 ...     -1.0            
1237662225166435295 249.043432111 20.2536057608 ...     -1.0            
1237662225166369213 248.961398717 20.3623116758 ...     -1.0

In [15]:
odysats = allsats[allsats['HOST_NSAID']==147100]
odysats

<Table masked=True length=8>
       objID              ra           dec      ... ZQUALITY SPEC_REPEAT
       int64           float64       float64    ... float64   string384 
------------------- ------------- ------------- ... -------- -----------
1237662662147571761 248.393225697 19.9461400824 ...      4.0        SDSS
1237662224092364842 247.877387573 20.0936248407 ...      4.0        SDSS
1237662698115432783 248.152079686 19.8102591366 ...      4.0            
1237662698115432544 248.051339587 19.6957396993 ...      4.0            
1237662224092496776 248.195368592 19.8670126788 ...      3.0            
1237662698115433445  248.16342684  19.792207594 ...      4.0            
1237662662147703369  248.58079685 19.7200349023 ...      4.0            
1237662662147310256 247.825892241 20.2108787761 ...      4.0

## Are the objID-matches complete?

In [20]:
np.all(np.in1d(odysats['objID'], mltab['OBJID_1']))

True

OK, all sats are represented, at least

In [22]:
np.sum(np.in1d(odycat['objID'], mltab['OBJID_1'])), np.sum(np.in1d(odycat['objID'], mltab['OBJID_2'])), len(odycat)

(34354, 20972, 116533)

In [25]:
np.sum(np.in1d(mltab['OBJID_1'], odycat['objID'])), np.sum(np.in1d(mltab['OBJID_2'], odycat['objID']))

(34337, 20955)

In [36]:
mlclosemsk = SkyCoord(mltab['RA'], mltab['DEC'], unit=u.deg).separation(ody.coords) < 2*u.deg
np.sum(np.in1d(mltab[mlclosemsk]['OBJID_1'], odycat['objID'])), np.sum(np.in1d(mltab[mlclosemsk]['OBJID_2'], odycat['objID'])), np.sum(mlclosemsk)

(34337, 20955, 52841)

Lets just forge ahead assuming OBJID_1 is the thing to match against

In [52]:
mltab['objID'] = mltab['OBJID_1']
joinedcat = table.join(odycat, mltab)
joinedsats = table.join(odysats, mltab)
del mltab['objID']
joinedcat

<Table masked=True length=34354>
       objID              ra      ... BEST_GUESS_CLASS_WISE
       int64           float64    ...         int64        
------------------- ------------- ... ---------------------
1237662223555166384  247.11293002 ...                     0
1237662223555166389 247.099255456 ...                    --
1237662223555166535 247.071365106 ...                     0
1237662223555166681 247.117228848 ...                     0
1237662223555166763 247.079130398 ...                     0
1237662223555166765  247.05662298 ...                     0
1237662223555166768 247.070892702 ...                     0
1237662223555166777 247.068689264 ...                     0
1237662223555166928 247.093064042 ...                    --
1237662223555166931 247.086728605 ...                    --
                ...           ... ...                   ...
1237665232670950000 247.516330652 ...                    --
1237665232670950004 247.543981178 ...                    --
1237665232670950071 247.484866959 ...                    --
1237665232670950162 247.509678476 ...                    --
1237665232670950180 247.518740755 ...                    --
1237665232670950264 247.548072545 ...                    --
1237665232670950287 247.548182055 ...                     0
1237665232670950416 247.598368873 ...                    --
1237665232671015097 247.628492036 ...                     0
1237665232671015571 247.621859146 ...                     0

In [344]:
def find_similar_objects(baseobj, catalog, propertytolerances):
    """
    minus sign on propertytolerances means *multiplicative* tolerance (>1).  Can also
    be a function that takes the base value and comparison vals and gives a mask
    """
    msk = np.ones(len(catalog), dtype=bool)
    
    for prop, tol in propertytolerances.items():
        if '-' in prop:
            m1, m2 = prop.split('-')
            compval = catalog[m1] - catalog[m2]
            baseval = baseobj[m1] - baseobj[m2]
        else:
            compval = catalog[prop]
            baseval = baseobj[prop]
            
        if callable(tol):
            msk = msk & tol(baseval, compval)
        elif tol >=0:
            msk = msk & (np.abs(compval - baseval) < tol)
        else:
            msk = msk & ((baseval/-tol)<compval) & (compval<(baseval*-tol))
    
    return catalog[msk]

def show_similar_objects(baseobj, catalog, propertytolerances, nameinfo=None, sorton=None):
    simcat = find_similar_objects(baseobj, catalog, propertytolerances)
    ras = np.insert(simcat['ra'], 0, baseobj['ra'])
    decs = np.insert(simcat['dec'], 0, baseobj['dec'])
    if nameinfo is None:
        nameinfos = []
    else:
        nameinfos = [nameinfo] if isinstance(nameinfo, basestring) else nameinfo
        
    names = []
    if sorton is not None:
        if sorton.startswith('-'):
            sorton = sorton[1:]
            reversesort = True
        else:
            reversesort = False
        sortvals = []
    enum = list(enumerate(simcat))
    enum.insert(0, (-1, baseobj))
    for i, row in enum:
        pieces = ['Realobj'] if i==-1 else ['Fakeobj' + str(i)]
        for nameinfo in nameinfos:
            label = 'p' if nameinfo.startswith('PROBABILITY') else nameinfo
            pieces.append('{}={:.3f}'.format(label, row[nameinfo]))
        names.append('_'.join(pieces))
        if sorton is not None:
            sortvals.append(row[sorton])
            
    if sortvals:
        sorti = np.argsort(sortvals)
        if reversesort:
            sorti = sorti[::-1]
        ras = ras[sorti]
        decs = decs[sorti]
        names = np.array(names)[sorti]

    targeting.sampled_imagelist(ras, decs, names=names, n=np.inf, posttoimglist=0.5)
    return simcat

In [346]:
tols = {'r':1, 'g-r':.2, 'petroR50_r':-3.5, 
               'spec_z':lambda objval, catval:(np.abs(catval-objval)/objval > .5) & (catval>-1),
               }#'PROBABILITY_CLASS_1': lambda objval, catval: catval <  (objval*.5)}

nonsatcats = []
for sat in joinedsats:
    nonsatcats.append(show_similar_objects(sat, joinedcat, tols, ['spec_z', 'PROBABILITY_CLASS_1'], 
                                           sorton='-PROBABILITY_CLASS_1'))

In [347]:
realsats_idxs = [2, 5, 7]
nonsat_idxs = [[1, 14], [12, 11], [1, 33]]

In [350]:
#convert those to rows
realsats_rows = joinedsats[np.array(realsats_idxs)]
nonsat_rows = [[nonsatcats[ri][li] for li in lis] for ri, lis in zip(realsats_idxs, nonsat_idxs)]

In [351]:
def row_to_cutout_img(row, size=(400, 400), scale=0.1):
    baseurl = 'http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx'
    res = requests.get(baseurl, dict(ra=row['ra'], dec=row['dec'], scale=scale, width=size[0], height=size[1]))
    sio = StringIO(res.content)
    sio.seek(0)
    return mpimg.imread(sio, 'jpeg')

In [359]:
fig, spls = plt.subplots(len(nonsat_idxs[0])+1, len(realsats_idxs), figsize=(10, 10))

for ax, row in zip(spls[0], realsats_rows):
    imdata = row_to_cutout_img(row)
    ax.imshow(imdata)
    ax.set_xticks([])
    ax.set_yticks([])

for imcol in range(len(nonsat_idxs[0])+1):
    for ax, row in zip(spls[1:, imcol], nonsat_rows[imcol]):
        imdata = row_to_cutout_img(row)
        ax.imshow(imdata)
        ax.set_xticks([])
        ax.set_yticks([])
        
spls[0, 0].set_ylabel('Real Satellites', fontsize=20)
for spl in spls[1:, 0]:
    spl.set_ylabel('Non-Satellites', fontsize=20)
fig.tight_layout()
plt.savefig('satnonsatcomp2.png')
plt.savefig('satnonsatcomp2.pdf')

In [362]:
!open .

In [360]:
print 'Real sats:'
for row in realsats_rows:
    print row['ra'], row['dec'], row['spec_z'],  row['PROBABILITY_CLASS_1']

Real sats:
247.825892241 20.2108787761 0.008145 0.459613790263
248.051339587 19.6957396993 0.008224 0.525459360418
248.16342684 19.792207594 0.007471 0.0140341939762


In [365]:
print 'Non-sats:'
for i,rows in enumerate(nonsat_rows):
    print 'For realsat #', i+1
    for row in rows:
        print row['ra'], row['dec'], row['spec_z'],  row['PROBABILITY_CLASS_1']

Non-sats:
For realsat # 1
248.282242282 20.7635522393 0.0358905 0.0245511012197
247.430414803 19.3932699725 0.0352353 0.0633943760791
For realsat # 2
247.586344578 20.5748995768 0.0369235 0.163668118206
247.827149304 19.9813131712 0.0633817 0.00217939800516
For realsat # 3
248.139305346 19.3855129623 0.215105 0.00238110292132
247.082860086 19.7554252371 2.72519 0.00057479534986


# Older version 

In [295]:
fig, spls = plt.subplots(len(lowprob_idxs[0])+1, len(realsats_idxs), figsize=(7, 12))

for ax, row in zip(spls[0], realsats_rows):
    imdata = row_to_cutout_img(row)
    ax.imshow(imdata)
    ax.set_xticks([])
    ax.set_yticks([])

for imcol in [0, 1]:
    for ax, row in zip(spls[2:, imcol], lowprob_rows[imcol]):
        imdata = row_to_cutout_img(row)
        ax.imshow(imdata)
        ax.set_xticks([])
        ax.set_yticks([])
        
spls[0, 0].set_ylabel('Real\nSatellites', fontsize=20)
spls[1, 0].set_ylabel('High-Prob\nNon-Satellites', fontsize=20)
fig.tight_layout()
spls[3, 0].set_ylabel('{0}Low-Prob\n{0}Non-Satellites'.format(' '*36), fontsize=20)
plt.savefig('satnonsatcomp.png')
plt.savefig('satnonsatcomp.pdf')